In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install roboflow

In [3]:
import math
import numpy as np
from roboflow import Roboflow

rf = Roboflow(api_key="v6urEc00Xf5zMsxFoaaj")
project = rf.workspace().project("side_pp")
model = project.version("1").model

job_id, signed_url, expire_time = model.predict_video(
    #"/content/drive/MyDrive/Colab-Notebooks/人工智慧實務/pp_videos/nv1.2.mp4",
    "/content/drive/MyDrive/人工智慧實務/pp_videos/nv1.2.mp4",
    fps=29,
    prediction_type="batch-video",
)

results = model.poll_until_video_results(job_id)

loading Roboflow workspace...
loading Roboflow project...
Checking for video inference results for job 647a9482-309d-484b-9869-4c7cc51f5d02 every 60s
(0s): Checking for inference results


In [4]:
frame_ball = [] # 紀錄每一幀下的球
time=0.0 # 記錄時間
cnt2=0

def sortT_x(dict):
  return dict['x']
def sortT_y(dict):
  return dict['y']
def arrange_LR(table):
  table.sort(key=sortT_x)
  x1,x2,x3,x4=0,0,0,0
  for i in range(len(table)):
    if i < len(table)/2:
      table[i]['p']="L"
      if x1<table[i]['x']:
        x1 = table[i]['x']
        x2 = x1 + table[i]['width']
    else:
      table[i]['p']="R"
      if x3<table[i]['x']:
        x3 = table[i]['x']
        x4 = x3 + table[i]['width']
  return x1,x2,x3,x4


for frame in range(len(results['frame_offset'])): # 每一幀
  flag = False # 紀錄是否有球
  cnt2+=1
  #print(cnt2)
  time+=results['side_pp'][frame]['time']#記錄時間
  #print(time)
  table=[]
  for obj in results['side_pp'][frame]['predictions']:
    if obj['class']=='table':
      table.append(obj)
  x1,x2,x3,x4 = arrange_LR(table)

  for obj in results['side_pp'][frame]['predictions']: # 該幀下的物件
    if obj['class'] == 'pp_ball': # 如果有球
      if obj['x']>=x1 and obj['x']<=x2 :
        obj['p'] = 'L'
      elif obj['x']>=x3 and obj['x']<=x4 :
        obj['p'] = 'R'
      obj['time'] = str(time)
      obj['frame'] = str(frame)
      frame_ball.append(obj.copy()) # 紀錄下來
      flag = True # 紀錄下來
      break # 跳開
  if not flag: # 如果沒有球
    frame_ball.append(None) # 塞一個 None
cnt=0
for sliding_point in range(2, len(frame_ball)): # 從第三幀開始
  for frame in range(sliding_point - 2, sliding_point + 1): # 三幀為一組
    flag = False # 紀錄是否有球
    if frame_ball[frame] == None: # 如果沒球
      flag = True # 紀錄下來
      break # 跳開

  if flag: # 若該組下有幀沒球
    continue # 就跳至下一組

  # 如果該組下的球, 在第二幀時低於第一幀時的高度
  if frame_ball[sliding_point - 1]['y'] > frame_ball[sliding_point - 2]['y'] and \
    frame_ball[sliding_point]['y'] < frame_ball[sliding_point - 1]['y']: # and \
    # abs(frame_ball[sliding_point]['x'] - frame_ball[sliding_point - 2]['x']) >= 50: # 且在第三幀時高於第二幀時的高度, 就代表球落地彈跳
    ball = frame_ball[sliding_point - 1]
    for i in range(len(table)):
      if table[i]['x'] - table[i]['width'] <= ball['x'] <= table[i]['x'] and table[i]['y'] - table[i]['height'] <= ball['y'] <= table[i]['y']:
        print(ball['x'], ball['y'], "第" + str(i) + "格")
        break
    cnt+=1

77.0 135.5 第1格
112.0 130.5 第1格
241.0 150.5 第2格
305.5 97.5 第2格
241.0 151.0 第2格
316.0 157.0 第2格
314.0 148.0 第2格
135.0 143.0 第2格
39.5 119.0 第1格
316.0 157.0 第2格
124.0 130.5 第2格
314.0 148.5 第2格
134.5 143.5 第2格
39.0 119.0 第1格
